# Multiple Linear Regression of RNA-Seq data in Aspergillus aculeatus

# Part2 Feature Selection

<strong>Objective:</strong><br />
1. ทำการคัดกรองยีน (feature/attribute) ที่ไม่เกี่ยวข้องกับ enzyme activity ออกไป
2. เลือกยีน (feature/attribute) ที่มีความสำคัญหรือเกี่ยวข้องกันกับค่า enzyme activity ที่มากที่สุดจำนวน 5-9 ยีน (จำนวน ยีนที่เลือกควรจะน้อยกว่าจำนวนตัวอย่าง)

บทความที่เกี่ยวข้องกับการทำ Feature Selection แบบอธิบายหลักการและ concept

https://www.facebook.com/dataforeverybody/posts/2363798550546954/

https://medium.com/@key.sompornpettakon.statkmitl/feature-selection-101-9eb8cf362dff

https://medium.com/@doohpim/03-feature-selection-%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B9%80%E0%B8%A5%E0%B8%B7%E0%B8%AD%E0%B8%81%E0%B8%84%E0%B8%B8%E0%B8%93%E0%B8%A5%E0%B8%B1%E0%B8%81%E0%B8%A9%E0%B8%93%E0%B8%B0-446f8f0c094b

https://arnondora.in.th/feature-selection-machine-learning/

In [1]:
#เริ่มต้นด้วยการ Load package ที่จำเป็นสำหรับการทำงาน
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#ตรวจสอบ version ของ package ที่ได้ทำการเรียกมา
print(f'pandas  version: {pd.__version__}')
print(f'numpy   version: {np.__version__}')
print(f'seaborn version: {sns.__version__}')

pandas  version: 0.25.3
numpy   version: 1.17.4
seaborn version: 0.9.0


In [3]:
#เริ่มต้นด้วยการโหลดข้อมูลจากขั้นตอนที่แล้วมา 
#(เนื่องจากข้อมูลมีจำนวนมาก บาง column อาจไม่ถูกแสดงน๊า)
url='https://raw.githubusercontent.com/wutto/aspergillusAculeatus/main/1_input_rapidMiner_8623_CMCase.csv'
df=pd.read_csv(url)
df.head(10)

,geneId,acu00000_1,acu00000_13,acu00000_14,acu00000_15,acu00000_2,acu00000_4,acu00000_5,acu00000_9,acu00002_2,...,acu08288_1,acu08289_1,acu08290_1,acu08291_1,acu08292_1,acu08293_1,acu08294_1,acu08295_1,activity,activityClass
0,BCC199__A,8.553751,0.172295,0.753331,16.922520,48.318485,0.344598,6.002934,1.195810,2.951030,...,3.948479,3.369402,1.049392,1.218003,9.117593,47.871967,4.597338,1.959886,1.516,medium
1,BCC199__B,17.619043,0.000000,0.281621,4.535475,15.585501,0.255006,2.996228,0.463476,2.474162,...,1.742349,0.475442,0.327337,0.000000,0.879822,79.582764,1.578946,0.556271,1.307,medium
2,BCC199__C,15.777807,0.000000,0.000000,4.093641,16.862034,0.078562,1.626894,0.569101,3.262403,...,1.640925,0.842889,0.492263,0.257300,6.860642,83.444359,3.561083,1.164043,1.575,medium
3,BCC56535__A,0.889026,0.234836,1.877119,12.957662,22.900455,3.847135,12.824106,0.761631,0.409413,...,5.997713,3.625029,15.687724,34.086227,30.144608,55.342480,6.240952,0.650502,0.260,low
4,BCC56535__B,0.000000,0.000000,0.000000,4.256706,12.697367,0.000000,0.384335,0.246236,0.000000,...,0.144612,0.074069,0.000000,1.020378,13.492409,65.686302,8.724833,0.138616,0.145,low
5,BCC56535__C,0.000000,0.000000,0.000000,7.068939,7.409282,0.213775,1.186643,0.000000,0.077722,...,0.000000,0.000000,0.000000,1.270019,7.693921,58.360996,5.185790,0.034781,0.195,low
6,BCC56535__D,0.057678,0.000000,1.021568,7.296792,8.579680,0.391414,5.521814,0.400340,0.000000,...,0.248023,0.158293,0.252113,3.693663,8.895016,69.126770,1.527307,0.000000,0.251,low
7,HUT2365__A,0.000000,0.220658,10.100449,9.381460,3.981732,2.054885,33.030998,1.245618,1.547278,...,2.275030,0.645219,12.719957,14.126668,98.608780,38.251125,2.338180,2.915910,5.333,high
8,HUT2365__B,0.581114,0.097028,3.222058,3.890594,7.513707,1.732018,33.175011,1.382367,1.077620,...,0.737417,0.776796,2.879683,9.945427,21.128397,29.774530,2.653734,3.243918,3.624,high
9,HUT2365__C,0.598314,0.094493,1.786440,2.082320,7.243462,1.473324,29.517067,1.538031,1.867690,...,1.015699,0.619233,2.761027,9.927620,19.668028,29.649534,2.786057,3.379966,4.050,high


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 8626 entries, geneId to activityClass
dtypes: float64(8623), int64(1), object(2)
memory usage: 674.0+ KB


## ทำการโหลด package sklearn: Classifier

In [5]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn import metrics

In [6]:
print(f'sklearn version: {sklearn.__version__}')

sklearn version: 0.22.1


In [7]:
df.columns

Index(['geneId', 'acu00000_1', 'acu00000_13', 'acu00000_14', 'acu00000_15',
       'acu00000_2', 'acu00000_4', 'acu00000_5', 'acu00000_9', 'acu00002_2',
       ...
       'acu08288_1', 'acu08289_1', 'acu08290_1', 'acu08291_1', 'acu08292_1',
       'acu08293_1', 'acu08294_1', 'acu08295_1', 'activity', 'activityClass'],
      dtype='object', length=8626)